<a href="https://colab.research.google.com/github/evanphilipsmith/infant-cry-prediction/blob/main/evaluate_checkpoint.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Dependencies and Imports

In [ ]:
![ ! -f utils.py ] && wget https://raw.githubusercontent.com/evanphilipsmith/infant-cry-prediction/main/utils.py
![ ! -f dataset.py ] && wget https://raw.githubusercontent.com/evanphilipsmith/infant-cry-prediction/main/dataset.py
![ ! -f network.py ] && wget https://raw.githubusercontent.com/evanphilipsmith/infant-cry-prediction/main/network.py
![ ! -f pretrained_model.pth ] && wget https://raw.githubusercontent.com/evanphilipsmith/infant-cry-prediction/main/pretrained_model.pth
!pip install torch==1.7.1+cu101 torchvision==0.8.2+cu101 torchaudio==0.7.2 -f https://download.pytorch.org/whl/torch_stable.html
![ ! -d donateacry-corpus ] && git clone https://github.com/gveres/donateacry-corpus.git

In [1]:
import torch
from torch.nn import CrossEntropyLoss
from torch.utils.data import DataLoader
from tqdm import tqdm
import numpy as np
from sklearn.metrics import f1_score

from utils import load_checkpoint, evaluate
from dataset import DonateACryDataset

assert torch.cuda.is_available(), "Select a GPU from Runtime -> Change runtime type"

# Evaluate Checkpoint

In [2]:
clean_dirs = ['./donateacry-corpus/donateacry_corpus_cleaned_and_updated_data/']
clean_test_data = DonateACryDataset(clean_dirs, train=False)

In [7]:
batch_size = 1
clean_test_loader = DataLoader(clean_test_data, batch_size=batch_size, shuffle=False, num_workers=2, pin_memory=True)

loss = CrossEntropyLoss()
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
bar = tqdm(total=len(clean_test_loader), position=0, leave=True)

saved_model = load_checkpoint('pretrained_model.pth', dir='./').cuda()

  0%|          | 0/92 [00:00<?, ?it/s]

In [8]:
losses, accuracies, preds = evaluate(saved_model, clean_test_loader, loss, device, bar)

evaluating: 100%|██████████| 92/92 [00:36<00:00,  2.73it/s]

In [9]:
print('num epochs trained', saved_model.num_epochs_trained)
print('micro:', f1_score(clean_test_data.data.target, preds, average='micro'))
print('macro:', f1_score(clean_test_data.data.target, preds, average='macro'))
print('distribution of preds:', np.unique(preds, return_counts=True))

num epochs trained 136
micro: 0.8478260869565218
macro: 0.6595959595959596
distribution of preds: (array([0, 1, 2, 3, 4]), array([ 7,  4, 77,  1,  3]))
